In [1]:
import sktime
import tslearn
import os
import numpy as np

In [2]:
### get current path
from pathlib import Path
path = Path.cwd()
path

PosixPath('/Users/mojtabaaskarzadeh/Library/CloudStorage/GoogleDrive-ma00048@mix.wvu.edu/My Drive/1-time-series classification in manufacturing/Code')

In [3]:
## Load the dataset
path_dir = '/Users/mojtabaaskarzadeh/Library/CloudStorage/GoogleDrive-ma00048@mix.wvu.edu/My Drive/1-time-series classification in manufacturing/Datasets/'
DSname = 'PHM2022'
path = path_dir + "/" + DSname + "/"
X_train = np.load(path + 'X_train.npy')
X_test = np.load(path + 'X_test.npy')
y_train = np.load(path + 'y_train.npy').reshape((-1,))
y_test = np.load(path + 'y_test.npy').reshape((-1,))
ts = np.concatenate((X_train, X_test), axis=0)

print(f"\n The dataset shape is:{ts.shape}")
print(f"\n The number of data samples (N) is:{ts.shape[0]}")
print(f"\n The number of TS length (T) is:{ts.shape[1]}")
print(f"\n The number of TS dimention (M) is:{ts.shape[2]}")


 The dataset shape is:(53785, 749, 3)

 The number of data samples (N) is:53785

 The number of TS length (T) is:749

 The number of TS dimention (M) is:3


In [4]:
X_train = np.swapaxes(X_train, 1,2)
X_test = np.swapaxes(X_test, 1,2)

In [5]:
from pyts.multivariate.transformation import WEASELMUSE
from sklearn.pipeline import Pipeline
from sklearn.linear_model import RidgeClassifierCV


In [6]:
pipeline = [('transfomer', WEASELMUSE()), ('classifier', RidgeClassifierCV(alphas=np.logspace(-3, 3, 10)))] 

# pipeline object 
pipe = Pipeline(pipeline,verbose=True)

In [7]:
pipe.fit(X_train, y_train)
y_pred = pipe.predict(X_test)

[Pipeline] ........ (step 1 of 2) Processing transfomer, total= 1.5min


: 

: 

In [10]:
from sklearn.metrics import confusion_matrix, classification_report
cm = confusion_matrix(y_test, y_pred)
print(cm)
print(classification_report(y_test, y_pred))

[[ 7  1  1  1  3  1]
 [ 0 26  0  0  0  0]
 [ 1  0 18  0  1  0]
 [ 1  0  0 19  0  2]
 [ 2  1  0  0 13  1]
 [ 5  0  1  0  0 15]]
              precision    recall  f1-score   support

           0       0.44      0.50      0.47        14
           1       0.93      1.00      0.96        26
           2       0.90      0.90      0.90        20
           3       0.95      0.86      0.90        22
           4       0.76      0.76      0.76        17
           5       0.79      0.71      0.75        21

    accuracy                           0.82       120
   macro avg       0.80      0.79      0.79       120
weighted avg       0.82      0.82      0.82       120



In [28]:
from sktime.classification.dictionary_based import MUSE

In [29]:
clf = MUSE(window_inc=4, use_first_order_differences=False)

In [30]:
clf.fit(X_train, y_train)
y_pred = clf.predict(X_test)

/Library/Frameworks/Python.framework/Versions/3.9/lib/python3.9/site-packages/sklearn/linear_model/_base.py:148: FutureWarning: 'normalize' was deprecated in version 1.0 and will be removed in 1.2. Please leave the normalize parameter to its default value to silence this warning. The default behavior of this estimator is to not do any normalization. If normalization is needed please use sklearn.preprocessing.StandardScaler instead.
  warnings.warn(


In [31]:
from sklearn.metrics import confusion_matrix, classification_report
cm = confusion_matrix(y_test, y_pred)
print(cm)
print(classification_report(y_test, y_pred))

[[19  0]
 [ 7  0]]
              precision    recall  f1-score   support

         0.0       0.73      1.00      0.84        19
         1.0       0.00      0.00      0.00         7

    accuracy                           0.73        26
   macro avg       0.37      0.50      0.42        26
weighted avg       0.53      0.73      0.62        26



/Library/Frameworks/Python.framework/Versions/3.9/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1334: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/Library/Frameworks/Python.framework/Versions/3.9/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1334: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/Library/Frameworks/Python.framework/Versions/3.9/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1334: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _wa

In [ ]:
classifier = ROCKETClassifier(n_jobs=-1)


In [ ]:
from sklearn.metrics import confusion_matrix, classification_report
cm = confusion_matrix(y_test, y_pred)
print(cm)
print(classification_report(y_test, y_pred))

##  Input "n" series with "d" dimensions of length "m" . default config  based on [4] is : Default (trees: n_estimators = 500, intervals: n_intervals = sqrt(m) × sqrt(d), att_subsample_size = 8 attributes per tree)

In [ ]:
## Implement the CanonicalIntervalForest classifier
from sktime.classification.interval_based import CanonicalIntervalForest
classifier = CanonicalIntervalForest(n_estimators=5, n_intervals=2, att_subsample_size=2, n_jobs=-1)
classifier.fit(X_train, y_train)

In [ ]:
y_pred = classifier.predict(X_test)

In [ ]:
from sklearn.metrics import confusion_matrix, classification_report
cm = confusion_matrix(y_test, y_pred)
print(cm)
print(classification_report(y_test, y_pred))

In [ ]:
## Implement the classifier on the second dimention
from sktime.classification.distance_based import KNeighborsTimeSeriesClassifier
classifier = KNeighborsTimeSeriesClassifier(distance='dtw')
classifier.fit(X_train[:,:,1], y_train)
y_pred = classifier.predict(X_test[:,:,1])

In [ ]:
cm = confusion_matrix(y_test, y_pred)
print(cm)
print(classification_report(y_test, y_pred))